In [1]:
!pip install acnportal 
# !pip install Mosek


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


## Imports

In [2]:
from acnportal.acnsim.events import GaussianMixtureEvents
from acnportal import acndata, acnsim
import pytz 
from datetime import datetime
import adacharge
from adacharge import *
from acnportal.acnsim.analysis import (proportion_of_energy_delivered,
                                       total_energy_delivered,
                                       total_energy_requested,
                                       proportion_of_demands_met,
                                       energy_cost,
                                       )

from acnportal.signals.tariffs.tou_tariff import TimeOfUseTariff
import pandas
# what is demand charge?   demand charge = peak demand * demand charge rate 

from acnportal.acnsim import Simulator
# from acnportal.acnsim import _increase_width


from acnportal.acnsim.network import ChargingNetwork
from acnportal.contrib.acnsim.network import StochasticNetwork

from acnportal import algorithms
from copy import deepcopy
import pandas as pd 
import numpy as np 
import cvxpy
from cvxpy import ECOS, MOSEK

## Experimental setup

In [3]:
# Timezone of the ACN we are using.
timezone = pytz.timezone('America/Los_Angeles')
API_KEY = 'DEMO_TOKEN'

# Start and End times are used when collecting data.
start = timezone.localize(datetime(2018, 9, 5))
end = timezone.localize(datetime(2018, 9, 7))

# Calculate the difference
difference_in_days = (end - start).days

print("Difference in days:", difference_in_days)



# How long each time discrete time interval in the simulation should be.
period = 5  # minutes

# Voltage of the network.
voltage = 200  # volts
transformer_cap = 50
# Default maximum charging rate for each EV battery.
default_battery_power = 32 * voltage / 1000 # kW

# Identifier of the site where data will be gathered.
site = 'caltech'


client = acndata.DataClient(API_KEY)

SAMPLES_PER_DAY = [100] * difference_in_days

print('installed solvers: ',cp.installed_solvers())
# default_battery_power = 32 * voltage / 1000  # kW
def level_2_network(transformer_cap=200, voltage=208, evse_per_phase=34):
    """ Configurable charging network for level-2 EVSEs connected line to line
        at 208 V.

    Args:
        transformer_cap (float): Capacity of the transformer feeding the network
          [kW]
        evse_per_phase (int): Number of EVSEs on each phase. Total number of
          EVSEs will be 3 * evse_per_phase.

    Returns:
        ChargingNetwork: Configured ChargingNetwork.
    """
    network = StochasticNetwork(early_departure=True)
    voltage = 208
    evse_type = 'AeroVironment'

    # Define the sets of EVSEs in the Caltech ACN.
    AB_ids = ['AB-{0}'.format(i) for i in range(evse_per_phase)]
    BC_ids = ['BC-{0}'.format(i) for i in range(evse_per_phase)]
    CA_ids = ['CA-{0}'.format(i) for i in range(evse_per_phase)]

    # Add Caltech EVSEs
    for evse_id in AB_ids:
        network.register_evse(acnsim.get_evse_by_type(evse_id, evse_type), voltage, 30)
    for evse_id in BC_ids:
        network.register_evse(acnsim.get_evse_by_type(evse_id, evse_type), voltage, -90)
    for evse_id in CA_ids:
        network.register_evse(acnsim.get_evse_by_type(evse_id, evse_type), voltage, 150)

    # Add Caltech Constraint Set
    AB = acnsim.Current(AB_ids)
    BC = acnsim.Current(BC_ids)
    CA = acnsim.Current(CA_ids)

    # Define intermediate currents
    I3a = AB - CA
    I3b = BC - AB
    I3c = CA - BC
    I2a = (1 / 4) * (I3a - I3c)
    I2b = (1 / 4) * (I3b - I3a)
    I2c = (1 / 4) * (I3c - I3b)

    # Build constraint set
    primary_side_constr = transformer_cap * 1000 / 3 / 277
    secondary_side_constr = transformer_cap * 1000 / 3 / 120
    network.add_constraint(I3a, secondary_side_constr, name='Secondary A')
    network.add_constraint(I3b, secondary_side_constr, name='Secondary B')
    network.add_constraint(I3c, secondary_side_constr, name='Secondary C')
    network.add_constraint(I2a, primary_side_constr, name='Primary A')
    network.add_constraint(I2b, primary_side_constr, name='Primary B')
    network.add_constraint(I2c, primary_side_constr, name='Primary C')

    return network



# preco offline optimalizacia nejde pre stochasticke problemy? su neidealne modely? alebo zle priradenie nabijaciek?
# skusit ci to ide pre stochastic s rovnakym poctom EVSE ako EV a aj rovnakymi indexami
# pozriet sa blizsie na tie swapy ci su dobre alebo nie v realnom zivote
# skusit este pohladat ze ci oni pouzivaju to pre StochasticNetwork

# For this experiment we use the predefined CaltechACN network.
cn = acnsim.sites.caltech_acn(basic_evse=True,
                              voltage=voltage,
                              transformer_cap=transformer_cap,
                              # network_type=StochasticNetwork
                              )

# all_station_ids = [f'station_{i}' for i in range(difference_in_days*100)]
# cn = acnsim.sites.simple_acn(station_ids=all_station_ids,
#                               voltage=voltage,
#                               aggregate_cap=transformer_cap)
# cn.early_departure = True

# cn = level_2_network(transformer_cap=transformer_cap,
#                      voltage=voltage,
#                      evse_per_phase=34)


Difference in days: 2
installed solvers:  ['CLARABEL', 'ECOS', 'ECOS_BB', 'MOSEK', 'OSQP', 'SCIPY', 'SCS']


In [4]:


gmm_events = GaussianMixtureEvents(n_components=20)
# # what is n_components? The number of mixture components.
sessions = list(client.get_sessions_by_time(site, start, end, timeseries=False))
gmm_events.fit(sessions)
# events = gmm_events.generate_events(sessions_per_day=SAMPLES_PER_DAY, period=period, voltage=voltage, max_battery_power=default_battery_power)
events = acnsim.acndata_events.generate_events(API_KEY, site, start, end, period, voltage, default_battery_power)

# gaussian mixture models inherits from stochastic models
# arguments of generate events = sessions_per_day:list[int] (number of sessions to sample for each day), period:float, voltage:float,
# max_battery_power:float
print(len(events))
# for ev in events.queue():
#     print(ev)





C:\Users\OMI\AppData\Local\Programs\Python\Python312\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
found 0 physical cores < 1
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "C:\Users\OMI\AppData\Local\Programs\Python\Python312\Lib\site-packages\joblib\externals\loky\backend\context.py", line 282, in _count_physical_cores
    raise ValueError(f"found {cpu_count_physical} physical cores < 1")


186


In [5]:
simulations = []
labels = []
signals = {'tariff': TimeOfUseTariff('sce_tou_ev_4_march_2019')}
# there are like 4-5 tariff schedules, but there are some functions to get schedule



## Model Predictive Control

In [6]:
max_energy_obj = [
    adacharge.ObjectiveComponent(total_energy),
                    adacharge.ObjectiveComponent(adacharge.quick_charge),
                    adacharge.ObjectiveComponent(adacharge.equal_share, 1e-12)
]
mpc = adacharge.AdaptiveSchedulingAlgorithm(max_energy_obj, solver="ECOS")
# mpc = adacharge.AdaptiveSchedulingAlgorithm(max_energy_obj, solver="MOSEK")
mpc2 = adacharge.AdaptiveSchedulingAlgorithm(max_energy_obj, solver="SCS")


# -> optimiser MOSEK not installed fix this problem


# look for another solvers that can stop the warning to be shown 


# mpc_sp_sim, mpc_tp_sim = experiment(mpc)

In [7]:
uncontrolled = algorithms.SortedSchedulingAlgo(algorithms.UncontrolledCharging())
simunc = acnsim.Simulator(deepcopy(cn), uncontrolled,deepcopy(events), start, period=period, verbose=False, signals=signals )

# labels.append('Uncontrolled charging')
# simulations.append(simunc)


In [8]:
llf = algorithms.SortedSchedulingAlgo(algorithms.least_laxity_first)
# station id sa prideluje od 0 do nekonecna 
simllf = acnsim.Simulator(deepcopy(cn), llf, deepcopy(events), start, period=period, verbose=False, signals=signals )
simllf.run()


labels.append('LLF with binary search')
simulations.append(simllf)



In [9]:
sim1 = acnsim.Simulator(deepcopy(cn), mpc,deepcopy(events), start, period=period, verbose=False, signals=signals )
# # print(sim1.get_active_evs())
# 
# print('before',mpc.interface.infrastructure_info()._station_ids_dict )
sim1.run()
# print('after',mpc.interface.infrastructure_info()._station_ids_dict)
# 
# def _increase_width(a, target_width):
#     """ Returns a new 2-D numpy array with target_width number of columns, with the contents
#     of a up to the first a.shape[1] columns and 0's thereafter.
# 
#     Args:
#         a (numpy.Array): 2-D numpy array to be expanded.
#         target_width (int): desired number of columns; must be greater than number of columns in a
#     Returns:
#         numpy.Array
#     """
#     if target_width <= a.shape[1]:
#         return a
#     new_matrix = np.zeros((a.shape[0], target_width))
#     new_matrix[:, : a.shape[1]] = a
#     return new_matrix
# 
# 
# # TODO prepisat funkciu run a zistit kde je chyba
# def run2(sim:acnsim.Simulator):
#      if sim.scheduler is None:
#         raise TypeError("Add a scheduler before attempting to call" " run().")
#      
#      while not sim.event_queue.empty():
#             current_events = sim.event_queue.get_current_events(sim._iteration)
#             # print('type  = ',type(current_events))
#             # print('current events', current_events)
#             # for event_car in current_events:
#             #     ...
#                 # print(type(event_car))
#                 # print('before',event_car.ev.station_id)
#             
#             
#             for e in current_events:
#                 sim.event_history.append(e)
#                 sim._process_event(e)
#                 # print('after: ',e.ev.station_id)
#             if (
#                 sim._resolve
#                 or sim.max_recompute is not None
#                 and (
#                     sim._last_schedule_update is None
#                     or sim._iteration - sim._last_schedule_update
#                     >= sim.max_recompute
#                 )
#             ):
#                 new_schedule = sim.scheduler.run()
#                 sim._update_schedules(new_schedule)
#                 if sim.schedule_history is not None:
#                     sim.schedule_history[sim._iteration] = new_schedule
#                 sim._last_schedule_update = sim._iteration
#                 sim._resolve = False
#             if not sim.event_queue.empty():
#                 width_increase = sim.event_queue.get_last_timestamp() + 1
#             else:
#                 width_increase = sim._iteration + 1
#             sim.pilot_signals = _increase_width(sim.pilot_signals, width_increase)
#             sim.charging_rates = _increase_width(sim.charging_rates, width_increase)
#             sim.network.update_pilots(sim.pilot_signals, sim._iteration, sim.period)
#             sim._store_actual_charging_rates()
#             sim.network.post_charging_update()
#             sim._iteration = sim._iteration + 1
# run2(sim=sim1)
# 
# stats = mpc.solve_stats
# overall_solve_time = 0
# all_sessions = 0
# for diction in stats:
#     overall_solve_time += diction['solve_time']
#     # all_sessions += diction['active_sessions']
#     
# print(overall_solve_time)
# print(all_sessions)
# events.to_json('anc.json')
# 
labels.append('MPC with convex optimisation solver')
simulations.append(sim1)
# try mosek solver? or is it only with offline optimisation


C:\Users\OMI\AppData\Local\Programs\Python\Python312\Lib\site-packages\cvxpy\problems\problem.py:1403: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


In [10]:
sim4 = acnsim.Simulator(deepcopy(cn), mpc2,deepcopy(events), start, period=period, verbose=False, signals=signals )
sim4.run()
labels.append('MPC with convex optimisation solver')
simulations.append(sim4)



C:\Users\OMI\AppData\Local\Programs\Python\Python312\Lib\site-packages\acnportal\acnsim\simulator.py:277: UserWarning: Invalid schedule provided at iteration 85. Max violation is 0.00138583749175325 A on Primary B at time index 7.
  warnings.warn(
C:\Users\OMI\AppData\Local\Programs\Python\Python312\Lib\site-packages\acnportal\acnsim\simulator.py:277: UserWarning: Invalid schedule provided at iteration 90. Max violation is 0.0004325889797058835 A on Primary B at time index 12.
  warnings.warn(
C:\Users\OMI\AppData\Local\Programs\Python\Python312\Lib\site-packages\acnportal\acnsim\simulator.py:277: UserWarning: Invalid schedule provided at iteration 93. Max violation is 0.004000438834211195 A on AV Pod at time index 9.
  warnings.warn(
C:\Users\OMI\AppData\Local\Programs\Python\Python312\Lib\site-packages\acnportal\acnsim\simulator.py:277: UserWarning: Invalid schedule provided at iteration 95. Max violation is 0.0014132857418047706 A on Primary B at time index 38.
  warnings.warn(
C:\U

In [11]:
def configure_sim(
    alg, 
    cap, 
    start, 
    events, 
    basic_evse=True,
    estimate_max_rate=False, 
    uninterrupted_charging=False,
    quantized=False,
    allow_overcharging=False,
    tariff_name=None,
    offline=False
):
    """ Configure simulation. """
    # start_time = TIMEZONE.localize(datetime.strptime(start, '%m-%d-%Y'))
        
    if estimate_max_rate:
        alg.max_rate_estimator = algorithms.SimpleRampdown()
        alg.estimate_max_rate = True
    alg.uninterrupted_charging = uninterrupted_charging
    alg.allow_overcharging = allow_overcharging

    # Some algorithms support a quantized option
    if quantized:
        try:
            alg.quantize = True
        except:
            pass
        try:
            alg.reallocate = True
        except:
            pass

    cn = acnsim.sites.caltech_acn(voltage=voltage,
                                  transformer_cap=cap,
                                  basic_evse=basic_evse,
                                  network_type=StochasticNetwork)
    if tariff_name is not None:
        signals = {'tariff': TimeOfUseTariff(tariff_name)}
    else:
        signals = {}
    sim = acnsim.Simulator(cn, alg, events, start, signals=signals,
                           period=period, verbose=False)
    
    if offline:
        alg.register_events(events)
        alg.solve()
    
    return sim

In [12]:
optimal_obj = [adacharge.ObjectiveComponent(total_energy)]
optimal_alg = adacharge.AdaptiveChargingAlgorithmOffline(optimal_obj, solver='ECOS')
# events2 = deepcopy(events)
# currently not working probably due to stochastically generated data
# # optimal_alg.run()
# 

experiment_events = deepcopy(events)
sim2 = acnsim.Simulator(deepcopy(cn), optimal_alg,experiment_events, start, period=period, verbose=False, signals=signals )

optimal_alg.register_events(experiment_events)
optimal_alg.solve()
sim2.run()

labels.append('Offline MPC')
simulations.append(sim2)





In [13]:
# while not events.empty():
#     e = events.get_event()
# events_special = events.get_current_events(events.get_last_timestamp())
# dictionar = []

# print(len(events))
# print(events.get_last_timestamp())
# for e in experiment_events.queue:
#     # print('event: ',e)
#     sim2._process_event(e[1])
# for e in events_special:
#     print('aaa')
#     sim2._process_event(e)
#     dictionar.append(e.ev.session_id)
    
# print(dictionar)
    
# sim = configure_sim(
#         cap=transformer_cap,
#         alg=deepcopy(optimal_alg),
#         start=start,
#         events=events,
#         basic_evse=True,
#         estimate_max_rate=False,
#         uninterrupted_charging=False,
#         quantized=False,
#         tariff_name='sce_tou_ev_4_march_2019',
#         offline=True
#     )

# print(sim2.get_active_evs())
# optimal_alg.
# optimal_alg.register_events(experiment_events)
y = optimal_alg.sessions
session_ids = []
for i in y:
    session_ids.append(i.session_id)
print('all session ids: ', session_ids)
# 
# 
# # print(y)
# 
# 
x = optimal_alg.interface.infrastructure_info()._station_ids_dict 
print(x)
# 
# print(x.station_ids)
# # # optimal_alg.register_interface(deepcopy(acnsim))
# 
# # station ids: i key-value
# print(x._station_ids_dict)
# 

all session ids:  ['2_39_79_383_2018-09-05 07:01:52.123265', '2_39_78_367_2018-09-05 11:54:26.985893', '2_39_79_378_2018-09-05 13:26:05.981247', '2_39_79_380_2018-09-05 13:49:44.986297', '2_39_79_381_2018-09-05 14:00:03.775977', '2_39_95_27_2018-09-05 14:07:58.875615', '2_39_78_366_2018-09-05 14:09:36.103199', '2_39_79_376_2018-09-05 14:27:02.561665', '2_39_138_29_2018-09-05 14:33:26.966732', '2_39_139_28_2018-09-05 14:34:05.345622', '2_39_79_382_2018-09-05 14:34:36.003068', '2_39_79_379_2018-09-05 14:48:31.756291', '2_39_79_377_2018-09-05 14:55:16.057671', '2_39_78_361_2018-09-05 14:57:21.868805', '2_39_88_24_2018-09-05 15:11:45.841374', '2_39_78_364_2018-09-05 15:14:59.966906', '2_39_130_31_2018-09-05 15:26:44.071246', '2_39_78_365_2018-09-05 15:27:14.529170', '2_39_129_17_2018-09-05 15:29:11.364122', '2_39_91_437_2018-09-05 15:30:41.640152', '2_39_128_18_2018-09-05 15:33:49.360725', '2_39_78_363_2018-09-05 15:33:54.732635', '2_39_127_19_2018-09-05 15:38:36.210722', '2_39_89_25_2018-

In [14]:
from python_implementation.mpc_sched import MyMpcAlgorithm

# MY IMPLEMENTATION 

quick_charge_obj = [
                    adacharge.ObjectiveComponent(total_energy),
                    adacharge.ObjectiveComponent(adacharge.quick_charge),
                    adacharge.ObjectiveComponent(adacharge.equal_share, 1e-12)
]
mpc = MyMpcAlgorithm(quick_charge_obj)



In [15]:
# proportion_llf = proportion_of_energy_delivered(sim=simllf)
# proportion_mpc = proportion_of_energy_delivered(sim=sim1)
# proportion_unc = total_energy_delivered(sim=simunc)
# print(proportion_llf)
# print(proportion_mpc)
# print(proportion_unc)
def get_simulation_stats(cn:StochasticNetwork, sim:acnsim.Simulator):
    res = {
        "energy delivered": sum(ev.energy_delivered for ev in sim.ev_history.values()),
        "proportion of energy delivered": proportion_of_energy_delivered(sim),
        "total energy delivered": total_energy_delivered(sim),
        "total energy requested":total_energy_requested(sim),
        "energy_cost":energy_cost(sim, tariff=signals["tariff"]),
        # "swaps": cn.swaps,
        # "never_charged": cn.never_charged,
        "demand_charge":acnsim.demand_charge(sim)
        }

    res['total_cost'] = res['energy_cost'] + res['demand_charge']
    if res['energy delivered'] != 0:
        res['$/kWh'] = res['total_cost'] / res['energy delivered']
    else:
        res['$/kWh'] = 0
    return res


sims_data = {}


for simulation in simulations:
    simulation_dict = get_simulation_stats(cn=cn,
                                           sim=simulation)
    for sim_key in simulation_dict.keys():
        if sims_data.get(sim_key, None) is None:  
            sims_data[sim_key] = [simulation_dict[sim_key]]
        elif sims_data.get(sim_key, None) is not None:  
            sims_data[sim_key].append(simulation_dict[sim_key])
print(sims_data)          
        



df2 = pd.DataFrame(sims_data, index = labels)
df2



{'energy delivered': [1336.5523913214283, 1505.791217770636, 1506.766128802646, 1512.799442059543], 'proportion of energy delivered': [0.8097224488369635, 0.9122522695013525, 0.9128428990594328, 0.9164980563257672], 'total energy delivered': [1336.5523913214283, 1505.791217770636, 1506.766128802646, 1512.799442059543], 'total energy requested': [1650.6302786111112, 1650.6302786111112, 1650.6302786111112, 1650.6302786111112], 'energy_cost': [190.36295313670257, 213.1301883942873, 213.1804322142935, 220.21777876687716], 'demand_charge': [745.231265625, 746.2252229650941, 746.2426558736485, 729.1284450476732], 'total_cost': [935.5942187617026, 959.3554113593814, 959.423088087942, 949.3462238145503], '$/kWh': [0.7000056449988431, 0.6371105104330019, 0.6367432010502844, 0.6275426850515619]}


,energy delivered,proportion of energy delivered,total energy delivered,total energy requested,energy_cost,demand_charge,total_cost,$/kWh
LLF with binary search,1336.552391,0.809722,1336.552391,1650.630279,190.362953,745.231266,935.594219,0.700006
MPC with convex optimisation solver,1505.791218,0.912252,1505.791218,1650.630279,213.130188,746.225223,959.355411,0.637111
MPC with convex optimisation solver,1506.766129,0.912843,1506.766129,1650.630279,213.180432,746.242656,959.423088,0.636743
Offline MPC,1512.799442,0.916498,1512.799442,1650.630279,220.217779,729.128445,949.346224,0.627543


In [16]:
# add a graph of proportion of  energy delivered in time 
# look at how their MPC settings work





# stochasticnetwork doesnt seem to work with offline MPC optimisation at all, it always gives bad results
